# Animations

In [ ]:
%pylab widget
#%pylab ipympl
#%pylab inline
from plotting_settings import *
from pathlib import Path

import os
cwd = os.getcwd()
plot_dir = os.path.join(cwd, 'plots')
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)
    
from ppmpy import ppm

import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

data_path = '/data/CoCo/'

In [ ]:
def smooth(arr, kernel):
    na = len(arr)
    nk = len(kernel)
    arr_padded = np.concatenate((np.mean(arr[:nk//2])*np.ones(nk),
                                 arr,
                                 np.mean(arr[-nk//2:])*np.ones(nk)))
    arr_padded = convolve(arr_padded, kernel, mode='same')
    
    return arr_padded[nk:-nk]

## Determine X1(t) in the convective layer as a reference

In [ ]:
rps = ppm.RprofSet(os.path.join(data_path, '1D-profiles/PPMSTAR/PPMSTAR-512/'), \
                   geometry='cartesian')
dumps = array(rps.get_dump_list())
yub = rps.bound_rad(dumps, 0., 3., var='X1', criterion='max_grad', \
                    return_var_scale_height=False)

x1_lcz = np.zeros(len(dumps))
for j, dmp in enumerate(dumps):
    rp = rps.get_dump(dmp)
    y = rp.get('Y')
    x1 = rp.get('X1')

    y_lcz = 1. + (2./3.)*(yub[j] - 1.)
    idx_lcz = np.argmin(np.abs(y - y_lcz))
    x1_lcz[j] = np.mean(x1[idx_lcz:])
    
ifig=1; close(ifig);
figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
kw = 16
kernel = np.ones(kw)/float(kw)
x1 = x1_lcz
x1_smooth = smooth(x1, kernel)
plot(x1, ls='-', color=lc[0], label='original')
plot(x1_smooth, ls='--', color=lc[1], label='smoothed')
legend(loc='lower right')
xlabel('Dump number')
ylabel('X1')
tight_layout()

## Plot all snapshots

In [ ]:
vort_ref = 1.8
vort_ref_str = r'{:.2f}'.format(vort_ref)
x1_fac = 10.
views = (
         {'q':'X1', 'sl':'Z-0.000', 'qlim':(0., 1.), 'cmap':'PuOr', 'lbl':r''},
         {'q':'A', 'sl':'Z-0.000', 'qlim':(-2.900, 2.999), 'cmap':'bwr',
          'lbl':r'$\left[ \left(A - \overline{A} \right) / A_\mathrm{rms} \right]_{\,z=0}$'},
         {'q':'VORT', 'sl':'Z-0.000', 'qlim':(0., 1.), 'cmap':'viridis',
          'lbl':r'$\tanh\left[ |\omega(z\,{=}\,0)| / '+vort_ref_str+r' \right]$'},
         {'q':'VY', 'sl':'Y-1.700', 'qlim':(-0.065, 0.065), 'cmap':'BrBG', 
          'lbl':r'$v_y(y\,{=}\,1.7)$'},
        )

codes = ('FLASH', 'MUSIC', 'PPMSTAR', 'PROMPI', 'SLH')
res0 = 256
runs = [c+'-'+str(res0) for c in codes] + ['PPMSTAR-512']
dump_interval = 5.

figwidth = 1.463705*pwidth
figheight = 0.95*pwidth
savedpi = 1080/figheight
print('Output resolution: {:.2f}x{:.2f}'.format(figwidth*savedpi, figheight*savedpi))

outpath = os.path.join(cwd, 'plots', 'animations', str(res0))
Path(outpath).mkdir(parents=True, exist_ok=True)

ifig=2; close(ifig);
fig = figure(figsize=(figwidth, figheight), dpi=0.75*scrdpi)
gs = mpl.gridspec.GridSpec(4, 7, width_ratios=[1, 1, 1, 1, 1, 1, 0.1])
for dmp in range(1, 401):
    clf()
    for ir, rid in enumerate(runs):
        for iv in range(4):
            tmp = rid.split('-')
            code = tmp[0]
            res = int(tmp[1])
            fn = '{:s}-{:04d}-{:s}-{:s}.npy'.format(rid, dmp,
                 views[iv]['q'], views[iv]['sl'])
            slice_path = os.path.join(data_path, '2D-slices', code, rid,
                         '{:04d}'.format(dmp), fn)
            data = np.load(slice_path)

            run_path = os.path.join(data_path, '1D-profiles', code, rid)
            rps = ppm.RprofSet(run_path, geometry='cartesian', verbose=0)
            rp = rps.get_dump(dmp)
            
            if views[iv]['sl'][0] in ('Y', 'Z'):
                data = np.transpose(data)[::-1,:]
            else:
                data = data[::-1,:]
            
            ax = subplot(gs[iv, ir])
            qmin = views[iv]['qlim'][0]
            qmax = views[iv]['qlim'][1]
            lbl = views[iv]['lbl']

            if views[iv]['q'] == 'VORT':
                data = np.tanh(data/vort_ref)
            if views[iv]['q'] == 'A':
                #adata = np.mean(data, axis=1)
                #adata = np.transpose(np.reshape(np.tile(adata, res), (res, res)))
                #rmsdata = np.mean((data - adata)**2, axis=1)**0.5
                #rmsdata = np.transpose(np.reshape(np.tile(rmsdata, res), (res, res)))
                #data = (data - adata)/rmsdata
                adata = rp.get('A')
                adata = np.transpose(np.reshape(np.tile(adata, res), (res, res)))
                rmsdata = rp.get('STDEV_A')
                rmsdata = np.transpose(np.reshape(np.tile(rmsdata, res), (res, res)))
                data = (data - adata)/rmsdata
            if views[iv]['q'] == 'X1':
                x1_min = x1_lcz[dmp] - 0.015
                x1_min = x1_min if x1_min > 0. else 0.
                data = (data - x1_min)/(1. - x1_min)
                data[data < 0.] = 0.
                data = np.tanh(x1_fac*data)

                x1_min_str = r'{:.3f}'.format(x1_min)
                x1_fac_str = r'{:.0f}'.format(x1_fac)
                lbl = r'$\tanh\left[ ' + x1_fac_str + r'(X_1 - '+x1_min_str+\
                      r') \,/\, (1 - ' + x1_min_str + r')  \right]_{\,z = 0}$'
        
            norm = mpl.colors.Normalize(vmin=qmin, vmax=qmax)
            cmap = views[iv]['cmap']
            pos = ax.imshow(data, cmap=cmap, norm=norm)

            ttl = r'{:s}, ${:d}^3$'.format(code, res)
            if iv == 0:
                ax.set_title(ttl, fontsize=textsize)
            ax.xaxis.set_ticks([])
            ax.yaxis.set_ticks([])

            fig.suptitle(r'$t = {:.0f}$'.format(dump_interval*dmp))

            if ir == len(runs) - 1:
                ax0 = subplot(gs[iv, -1])
                fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), \
                             cax=ax0, orientation='vertical', \
                             label=lbl)

    subplots_adjust(left=0.025, right=0.925, bottom=0.025, top=0.925, hspace=0.05, wspace=0.05)
    png_path = os.path.join(outpath, '2D-slices-{:d}-{:04d}.png'.format(res0, dmp))
    savefig(png_path, dpi=savedpi)
    print(png_path + ' written to disk.')